<a href="https://colab.research.google.com/github/Emanuel498/AprendoGit/blob/master/fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Ref.: https://www.kaggle.com/isaikumar/creditcardfraud/version/1
# Data frame with credit card data

from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1eBBx58FT36BFRTop06wrSF2x8t_Uh0eC',dest_path='./data/creditcard.csv')

df = pd.read_csv( './data/creditcard.csv' )

In [ ]:
# Details on data
print(df.shape)
print(df.columns)

# Compute correlation of features with respect to Class
# By default uses Pearson correlation that estimates linear correlation: 
# +1 or -1 linearly correlated, 0 non linear correlated
df.corr()["Class"].sort_values()

(284807, 31)
Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


V17      -0.326481
V14      -0.302544
V12      -0.260593
V10      -0.216883
V16      -0.196539
V3       -0.192961
V7       -0.187257
V18      -0.111485
V1       -0.101347
V9       -0.097733
V5       -0.094974
V6       -0.043643
Time     -0.012323
V24      -0.007221
V13      -0.004570
V15      -0.004223
V23      -0.002685
V22       0.000805
V25       0.003308
V26       0.004455
Amount    0.005632
V28       0.009536
V27       0.017580
V8        0.019875
V20       0.020090
V19       0.034783
V21       0.040413
V2        0.091289
V4        0.133447
V11       0.154876
Class     1.000000
Name: Class, dtype: float64

In [ ]:
# Extract features and labels

# drop column with Class
#X = df.drop(['Class'], axis=1)

#
# Obs. Modify the following code to select set of features
#
# The following line allows to select some variables to be used
X = df.loc[:,['V17','V10','Amount']]

# Class is the target to predict (classify)
y = df.Class

# Number of samples per class
unique, counts = np.unique(y, return_counts=True)
for i in range(0,len(unique)):
  print('Class %i: Samples %i' % (unique[i], counts[i]) )

# Number of attributes
print('Number of attributes: %i' % (X.shape)[1])

Class 0: Samples 284315
Class 1: Samples 492
Number of attributes: 3


In [ ]:
# Classifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Split train and test 
# stratify=y means the same % of classes is present in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0, stratify=y)

#
# Selection of classifier
#
#clf = KNeighborsClassifier(n_neighbors=5)
#clf = LogisticRegression(solver='lbfgs',max_iter=500)
clf = DecisionTreeClassifier()
#clf = GaussianNB()

# Train classifier
clf.fit(X_train,y_train)

# Priors GuassianNB
#print( clf.class_prior_ )

# Coefs Logistic Regression
# print( clf.coef_ )

# Confusion matrix
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

print(' Confusion matrix ------------------')
print(cm)

sum_diag = sum(cm[i][i] for i in range(2))
sum_all = sum(sum(cm))

print('\n % Corr ----------------------------')
print( sum_diag/sum_all*100 )


# Classification report
from sklearn.metrics import classification_report
target_names = ['1', '0']

print('\n Classification report ------------------')
report = classification_report(y_test, y_pred, target_names=target_names,output_dict=True)
print('Class 0 %s: ' % report['0'])
print('Class 1 %s: ' % report['1'])

# Amount of fraud detected
a = X_test.Amount
# TP 
ind = (y_pred == 1) & (y_test == 1)
# Amount for TP
amount_detected = np.sum( a[ ind ] ) 
# Total amount of fraud
ind = (y_test == 1)
amount_total = np.sum( a[ ind ] ) 

print('\n Amount ------------------ \n Detected = %f, Total = %f, Percentage = %f' % (amount_detected, amount_total, (amount_detected/amount_total)*100))

 Confusion matrix ------------------
[[142097     61]
 [    80    166]]

 % Corr ----------------------------
99.9009859273616

 Classification report ------------------
Class 0 {'precision': 0.7312775330396476, 'recall': 0.6747967479674797, 'f1-score': 0.7019027484143764, 'support': 246}: 
Class 1 {'precision': 0.999437321085689, 'recall': 0.9995708999845243, 'f1-score': 0.9995041060720629, 'support': 142158}: 

 Amount ------------------ 
 Detected = 18873.950000, Total = 30229.360000, Percentage = 62.435824


# Ejercicio

1.   Evaluar los clasificadores con el total de atributos y tomar nota de los indicadores de performance.
2.   Encontrar el mejor clasificador con 1 y 2 atributos. Justificar.
3.   ¿Cuál sería el peor clasificador?

Obs.: Para todos los casos anterior tomar nota de la matriz de confusión, el % de instancias correctamente clasificadas y el monto de fraude detectado.


